# Exploratory Notebook for bits & bobs

In [1]:
# Standard imports

import numpy as np
import pandas as pd
import json
import os

import matplotlib.pyplot as plt
import seaborn as snsb


## Intersection over Union

In [2]:
def return_IoU(box1, box2):

    """Function to generate the IoU for two bounding boxes in COCO format

    box1: list of floats representing bounding box (COCO format)
    box2: list of floats representing bounding box (COCO format)
    """

    # Convert from COCO format to corner format, this makes it simpler to calculate IoU
    box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]

    # Calculate intersection coordinates
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    # Check if there is no overlap
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou


In [35]:
# Get data path

currend_wd = os.getcwd()
target_file = "annotations.json"

raw_data_path = os.path.join(currend_wd, '..', 'raw_data')

# Load json file

with open(os.path.join(raw_data_path, target_file), 'r') as file:
    data = json.load(file)


In [4]:
data.keys()


dict_keys(['images', 'annotations', 'info', 'licenses', 'categories'])

In [75]:
subset_annotations = data['annotations'][:5]
test_bbox_0 = subset_annotations[0]['bbox']
test_bbox_1 = subset_annotations[1]['bbox']
test_bbox_2 = subset_annotations[2]['bbox']


test_bbox_0, test_bbox_1


([439.01, 187.01, 56.98000000000002, 53.98000000000002],
 [142.01, 185.01, 51.98000000000002, 54.98000000000002])

In [80]:
display(return_IoU(test_bbox_0, test_bbox_1))
# display(return_IoU(test_bbox_0, test_bbox_2))
# display(return_IoU(test_bbox_1, test_bbox_2))


0.0

In [28]:
data['annotations'][0].keys()


dict_keys(['category_id', 'image_id', 'id', 'iscrowd', 'color', 'area', 'segmentation', 'bbox', 'num_keypoints', 'keypoints', 'keypoints_3d', 'pose_variation', 'pose_category', 'exercise_category', 'cuboid_points', 'zaxis_rotation_deg', 'attire_top', 'attire_bottom', 'presenting_gender', 'waist_circumference', 'height', 'betas'])

In [ ]:
for annotation in annotations:

    annotation['iou']


In [34]:
keys_to_keep = ['image_id', 'category_id', 'id', 'bbox', 'pose_cateogory'
                , 'pose_variation', 'exercise_category', 'intersection_over_union']

annotations_cleaned = [
    {key: value for key, value in annotation.items() if key in keys_to_keep}
    for annotation in data['annotations']
]

# Specify your filename
target_save_file = 'annotations_cleaned.json'

# Writing to a file in the same directory
with open(os.path.join(raw_data_path,target_save_file), 'w') as file:
    json.dump(annotations_cleaned, file)

# Specify your filename
subset_filename = 'annotations_mini.json'


In [36]:

# Writing to a file in the same directory
with open(os.path.join(raw_data_path,subset_filename), 'w') as file:
    json.dump(annotations_cleaned[:20], file)


In [37]:
annotations_cleaned_mini = annotations_cleaned[:20]

annotations_cleaned_mini


[{'category_id': 0,
  'image_id': 1,
  'id': 1,
  'bbox': [439.01, 187.01, 56.98000000000002, 53.98000000000002],
  'pose_variation': 'situp',
  'exercise_category': ['HIIT']},
 {'category_id': 0,
  'image_id': 1,
  'id': 2,
  'bbox': [142.01, 185.01, 51.98000000000002, 54.98000000000002],
  'pose_variation': 'situp',
  'exercise_category': ['HIIT']},
 {'category_id': 0,
  'image_id': 1,
  'id': 3,
  'bbox': [302.01, 97.01, 38.98000000000002, 161.98000000000002],
  'pose_variation': 'tree_right',
  'exercise_category': ['Yoga']},
 {'category_id': 0,
  'image_id': 2,
  'id': 4,
  'bbox': [477.01, 113.01, 110.98000000000002, 91.98],
  'pose_variation': 'sideplank_right',
  'exercise_category': ['Yoga', 'HIIT']},
 {'category_id': 0,
  'image_id': 2,
  'id': 5,
  'bbox': [506.01, 99.01, 97.98000000000002, 239.98000000000002],
  'pose_variation': 'curls_right',
  'exercise_category': ['HIIT']},
 {'category_id': 0,
  'image_id': 2,
  'id': 6,
  'bbox': [56.01, 118.01, 44.98, 67.98],
  'pose_